<a href="https://colab.research.google.com/github/sajivhar4118/Rat-NOR/blob/main/Training_YoloV9_Public_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train and Run YOLOv9 on a Custom Dataset
#### Change file paths and programs as needed

## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [ ]:
!nvidia-smi

**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` constant.

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Clone and Install

**NOTE:** YOLOv9 is very new. At the moment, we recommend using a fork of the main repository. The `detect.py` script contains a bug that prevents inference. This bug is patched in the fork.

In [ ]:
!git clone https://github.com/SkalskiP/yolov9.git
%cd yolov9
!pip install -r requirements.txt -q

**NOTE:** Let's install the [`roboflow`](https://pypi.org/project/roboflow) package, which we will use to download our dataset from [Roboflow Universe](https://universe.roboflow.com/).

In [ ]:
!pip install -q roboflow

## Download model weights

**NOTE:** In the YOLOv9 paper, versions `yolov9-s` and `yolov9-m` are also mentioned, but the weights for these models are not yet available in the YOLOv9 [repository](https://github.com/WongKinYiu/yolov9).

In [ ]:
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt

In [ ]:
!ls -la {HOME}/weights

## Authenticate and Download the Dataset

**NOTE:** The dataset must be saved inside the `{HOME}/yolov9` directory, otherwise, the training will not succeed.

In [ ]:
%cd {HOME}/yolov9

In [ ]:
!pip install roboflow


#SNIPPIT FROM ROBOFLOW GOES HERE! Sample provided below:

from roboflow import Roboflow
rf = Roboflow(api_key="**************")
project = rf.workspace("ratnor").project("rat_nor")
version = project.version(6)
dataset = version.download("yolov9")


## Train Custom Model

In [ ]:
%cd {HOME}/yolov9

!python train.py \
--batch 16 --epochs 100 --img 480 --device 0 --min-items 0 --close-mosaic 15 \
--data /content/yolov9/RAT_NOR-6/data.yaml \
--weights {HOME}/weights/gelan-c.pt \
--cfg /content/yolov9/models/detect/gelan-c.yaml \
--hyp hyp.scratch-high.yaml




**NOTE:** By default, the results of each subsequent training sessions are saved in `{HOME}/yolov9/runs/train/`, in directories named `exp`, `exp2`, `exp3`, ... You can override this behavior by using the `--name` parameter.

In [ ]:
!ls {HOME}/yolov9/runs/train/

exp  exp2


### DOWNLOAD THE "Best.pt" file for future use! It saves time and GPU resources.

## Validate Custom Model

In [ ]:
%cd {HOME}/yolov9

!python val.py \
--img 640 --batch 32 --conf 0.001 --iou 0.7 --device 0 \
--data {dataset.location}/data.yaml \
--weights {HOME}/yolov9/runs/train/exp4/weights/best.pt

## Inference with Custom Model

In [ ]:
!python detect.py \
--img 480 --conf 0.5 --device 0 \
--weights /content/best.pt \
--source "/content/videos" \
--save-txt

In [ ]:
import os
import csv
import re

# Define screen dimensions
SCREEN_WIDTH = 852
SCREEN_HEIGHT = 480

# Function to convert relative coordinates to pixels
def to_pixel_coords(relative_x, relative_y):
    pixel_x = round(relative_x * SCREEN_WIDTH)
    pixel_y = round(relative_y * SCREEN_HEIGHT)
    return pixel_x, pixel_y

# Directory containing the text files
labels_dir = "/content/yolov9/runs/detect/exp5/labels"

# Directory where the CSV files will be saved
output_dir = "/content/csv/"
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# Loop over the range of sets (1 to 18)
for i in range(1, 19):
    # Create the output CSV file for this set
    output_file = os.path.join(output_dir, f"video_frames_R{i}.csv")

    # Create a list to store (frame_number, file_path) tuples
    frame_files = []

    # Process each text file
    for filename in os.listdir(labels_dir):
        if filename.startswith(f"NOR10_Acq_R{i}") and filename.endswith(".txt"):
            # Extract frame number from the end of the filename (digits before .txt)
            match = re.search(r'_(\d+)\.txt$', filename)
            if match:
                frame_number = int(match.group(1))  # Extract the frame number as an integer
                file_path = os.path.join(labels_dir, filename)
                frame_files.append((frame_number, file_path))

    # Sort the frame files by frame number
    frame_files.sort(key=lambda x: x[0])

    # Write header to the CSV file
    with open(output_file, mode='w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(["frame_number", "br_x", "br_y", "tl_x", "tl_y"])

        # Process sorted files
        for frame_number, file_path in frame_files:
            # Read the coordinates from the file
            with open(file_path, 'r') as f:
                lines = f.readlines()

                # Make sure there are enough lines to avoid index errors
                if len(lines) >= 2:
                    br_data = lines[0].split()
                    tl_data = lines[1].split()

                    # Extract relative coordinates
                    br_relative_x = float(br_data[1])
                    br_relative_y = float(br_data[2])
                    tl_relative_x = float(tl_data[1])
                    tl_relative_y = float(tl_data[2])

                    # Convert coordinates to pixels
                    br_x, br_y = to_pixel_coords(br_relative_x, br_relative_y)
                    tl_x, tl_y = to_pixel_coords(tl_relative_x, tl_relative_y)

                    # Write to CSV
                    csvwriter.writerow([frame_number, br_x, br_y, tl_x, tl_y])

    print(f"CSV file created: {output_file}")


### Download ALL the CSVs to your computer for futher processing


### Move on the next and final step: [Final-Data-Processing](https:https://github.com/sajivhar4118/Rat-NOR/blob/432363e24c67a7f3741660968ca048508be3c954/Final_Processing.md//)